In [2]:
import chess
import numpy as np

def one_hot_encode_piece(piece):
    pieces = list('rnbqkpRNBQKP.')
    arr = np.zeros(len(pieces))
    piece_to_index = {p: i for i, p in enumerate(pieces)}
    index = piece_to_index[piece]
    arr[index] = 1
    return arr

def encode_board(board):
    # first lets turn the board into a string
    board_str = str(board)
    # then lets remove all the spaces
    board_str = board_str.replace(' ', '')
    board_list = []
    for row in board_str.split('\n'):
        row_list = []
        for piece in row:
            row_list.append(one_hot_encode_piece(piece))
        board_list.append(row_list)
    return np.array(board_list)


# We'll use pandas to pull in all our training data
import pandas as pd

train_df = pd.read_csv('dataset/train.csv')

# We'll only use the first 10000 examples so things run fast,
# but you'll get better performance if you remove this line
# train_df = train_df[:20000]

# We'll also grab the last 1000 examples as a validation set
val_df = train_df[-1000:]
train_df.head()

import numpy as np
def encode_fen_string(fen_str):
    board = chess.Board(fen=fen_str)
    return encode_board(board)

# We'll stack all our encoded boards into a single numpy array
X_train = np.stack(train_df['board'].apply(encode_fen_string))
y_train = train_df['black_score']


X_val = np.stack(val_df['board'].apply(encode_fen_string))
y_val = val_df['black_score']

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten

# # With the Keras Sequential model we can stack neural network layers together
# model = Sequential([
#     Flatten(),
#     Dense(1024, activation='relu'),
#     Dense(1),
# ])

# model.compile(
#     optimizer='rmsprop',
#     loss='mean_squared_error')

# # To test things out, let's train for 20 epochs and see how our model is doing
# history = model.fit(
#     X_train,
#     y_train,
#     epochs=60,
#     validation_data=(X_val, y_val))
# # import tensorflow as tf
# # from tensorflow.keras.layers import Conv2D, Dense, Flatten, InputLayer
# # from tensorflow.keras.models import Sequential

# # # Define the input shape for the board representation
# # input_shape = (8, 8, 5)


# # # Define the CNN model
# # model = Sequential([
# #     InputLayer(input_shape=input_shape),
# #     Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
# #     Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
# #     Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
# #     Flatten(),
# #     Dense(256, activation='relu'),
# #     Dense(1)
# # ])

# # # Compile the model
# # model.compile(optimizer='adam', loss='mean_squared_error')


# # # random forrest
# # from sklearn.ensemble import RandomForestRegressor

# # model = RandomForestRegressor(n_estimators=100)

# # # Train the model on the training data
# # model.fit(X_train, y_train)

# import matplotlib.pyplot as plt

# # Lets plot the history of our training session to see how things progressed over time
# plt.style.use('ggplot')
# plt.plot(history.history['loss'], label='train loss')
# plt.plot(history.history['val_loss'], label='val loss')
# plt.legend()
# plt.title('Loss During Training')
# plt.show()

# def play_nn(fen, show_move_evaluations=False):
#     # We can create a python-chess board instance from the FEN string like this:
#     board = chess.Board(fen=fen)

#     # And then evaluate all legal moves
#     moves = []
#     input_vectors = []
#     for move in board.legal_moves:
#         # For each move, we'll make a copy of the board and try that move out
#         candidate_board = board.copy()
#         candidate_board.push(move)
#         moves.append(move)
#         input_vectors.append(encode_board(str(candidate_board)).astype(np.int32).flatten())
    
#     input_vectors = np.stack(input_vectors)
#     # This is where our model gets to shine! It tells us how good the resultant score board is for black:
#     scores = model.predict(input_vectors, verbose=0)
#     # argmax gives us the index of the highest scoring move
#     if board.turn == chess.BLACK:
#         index_of_best_move = np.argmax(scores)
#     else:
#         # If we're playing as white, we want black's score to be as small as possible, so we take argmax of the negative of our array
#         index_of_best_move = np.argmax(-scores)

#     if show_move_evaluations:
#         print(zip(moves, scores))
        
#     best_move = moves[index_of_best_move]

#     # Now we turn our move into a string, return it and call it a day!
#     return str(best_move)





import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, InputLayer
from tensorflow.keras.models import Sequential

# Define the input shape for the board representation
input_shape = (8, 8, 5)

# Define the CNN model
model = Sequential([
    InputLayer(input_shape=input_shape),
    Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

def encode_board(board_fen):
    # Code to encode the board FEN string into a 3D tensor with shape (8, 8, 5)
    ...

def play_nn(fen, show_move_evaluations=False):
    # Create a python-chess board instance from the FEN string
    board = chess.Board(fen=fen)
    
    # Evaluate all legal moves
    moves = []
    input_tensors = []
    for move in board.legal_moves:
        # Make a copy of the board and try the move out
        candidate_board = board.copy()
        candidate_board.push(move)
        moves.append(move)
        input_tensors.append(tf.convert_to_tensor([encode_board(str(candidate_board))]))
    
    # Stack the input tensors into a single tensor
    input_tensor = tf.concat(input_tensors, axis=0)
    
    # Use the model to predict the scores of each move
    scores = model.predict(input_tensor).squeeze()
    
    # Choose the best move based on the predicted scores
    if board.turn == chess.BLACK:
        index_of_best_move = tf.argmax(scores)
    else:
        index_of_best_move = tf.argmin(scores)
    
    best_move = moves[index_of_best_move]
    
    # Optionally print out the predicted scores for each move
    if show_move_evaluations:
        print(list(zip(moves, scores)))
        
    # Return the best move as a string
    return str(best_move)


In [3]:
test_df = pd.read_csv('dataset/test.csv')

test_df.head()

,id,board
0,7937,r1bqk2r/pp2bpp1/2n1pn1p/2pp4/3P1B2/2P1PN2/PP1N...
1,20035,2r2k1r/pp2pp1p/1q3npb/1B1N4/8/P4Q1P/1P3PP1/R2R...
2,71263,3b4/8/5k2/5p2/8/4K3/8/5B2 b - - 2 80
3,61997,5k2/R7/3r3p/2PP2pP/5pb1/P1K5/6B1/8 w - - 1 61
4,26510,r3r1k1/pb3p1p/1p1q2p1/3P1p2/3Q1P2/5N2/PP4PP/3R...


In [4]:
test_df['best_move'] = test_df['board'].apply(play_nn)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [5]:
submission = test_df[['id', 'best_move']]
submission.to_csv('submission.csv', index=False)